# 2. Get the Anecdote to Estimate Impact

Notebook to build all of the different stix objects associated with asking the reporting user about the Impact, and recording it through an Anecdote

We take v3.0 of the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc) specification, with particular focus on the Sighting Extensions for Evidence, and combine it with the standard Stix approach. There are nine stages:

1. First setup the global parameters and retrieve the context memory
2. Create the Anecdote object and connect it to the Identity,
2. Add these objects to the `ObservedData` object
4. Create the `Sighting` object with the `SightingAnecdote` extension
5. Create the Impact object
5. Create the next Task and its Sequence objects
6. Connect the Sequence object to the previous sequence object
6. Add all of the objects to the existing Incident object
7. Save the context memory


Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (This step also verifies the objects are created correctly, and is cool).



## A. Load Imports

### A.1 StixORM Imports


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage, ObservedData
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [3]:
import json
from Block_Families.Objects.SCO.Email_Message.make_email_msg import main as make_email_msg
from Block_Families.Objects.SRO.Relationship.make_sro import main as make_sro
from Utilities.local_make_general import invoke_save_options_context_block, invoke_save_incident_context_block, invoke_get_from_options_block, invoke_get_from_incident_block, invoke_move_unattached_to_other_block
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, invoke_make_indicator_block, invoke_make_event_block, invoke_make_sequence_block,
    invoke_make_task_block, invoke_make_incident_block, invoke_make_impact_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, invoke_make_url_block, invoke_make_e_msg_block, invoke_make_anecdote_block
)
from conv import conv

context_base = "../Orchestration/Context_Mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
alert_data = {
    "from": "evil@northkorea.nk",
    "to": "naive@example.com",
    "reporter": "Naive Smith",
    "url": "https://www.northkorea.nk/we/are/mad/"
}
alert_details = {
    "alert_name": "Suspcious Email",
    "alert_source": "user-report",
    "alert_log": "I have found a suspicious email"
}
anecdote_data = {
    "to": "naive@example.com"
}

TR_Context_Memory_Path = "./Context_Mem/Type_Refinery_Context.json"

### A.4 Retrieve the TR User Account, the Email Address, and the TR Identity from the Context Memory

and the previous task sequence

In [4]:
# 1. For Naive Smith, find the user account and the email. Plus get my own identity
reporter_name = alert_data["reporter"]
laptop_name = "HP Laptop 1"
TR_name = "Trusty Jones"
#
# 2. Setup variables and queries to get the objects from options
# get "HP Laptop 1"
context_type = {
    "context_type": "assets"
}
laptop_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": laptop_name,
        "comparator": "EQ"
    }
}
laptop_identity = invoke_get_from_options_block(laptop_ident_query, context_type, source_value=None, source_id=None)
# get reporter
context_type = {
    "context_type": "users"
}
reporter_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": reporter_name,
        "comparator": "EQ"
    }
}
reporter_identity = invoke_get_from_options_block(reporter_ident_query, context_type, source_value=None, source_id=None)
context_type = {
    "context_type": "me"
}
TR_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": TR_name,
        "comparator": "EQ"
    }
}
TR_identity = invoke_get_from_options_block(TR_ident_query, context_type, source_value=None, source_id=None)
#
print(f"\nreporter_identity-->{reporter_identity}")
print(f"\nlaptop_identity-->{laptop_identity}")
print(f"\nlaptop_identity-->{laptop_identity}")
print(f"\nTR_identity-->{TR_identity}")
#
# 3. save the objects to "unattached"
#
context_type = {
    "context_type": "unattached"
}
ident_results_obj_path = results_base + "/step0/Usr1__ident.json"
ident_results_context_path = results_base  + "/step1/context/usr1__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, context_type)
print(f" result->{result}")
ident_results_obj_path = results_base  + "/step0/TR_Identity__ident.json"
ident_results_context_path = results_base + "/step1/context/TR_Identity__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, context_type)
print(f" result->{result}")
ident_results_obj_path = results_base  + "/step0/Systems_Laptop1__ident.json"
ident_results_context_path = results_base + "/step1/context/Systems_Laptop1__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, context_type)
print(f" result->{result}")


reporter_identity-->{'type': 'identity', 'spec_version': '2.1', 'id': 'identity--0ff92ca8-5425-4649-bda9-37c564921944', 'created': '2024-04-27T06:06:06.901Z', 'modified': '2024-04-27T06:06:06.901Z', 'name': 'Naive Smith', 'description': 'A Naive Individual', 'roles': ['user', 'sales'], 'identity_class': 'individual', 'sectors': ['technology'], 'extensions': {'extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498': {'extension_type': 'property-extension', 'contact_numbers': [{'contact_number_type': 'work-phone', 'contact_number': '0499-999-109'}], 'email_addresses': [{'digital_contact_type': 'work', 'email_address_ref': 'email-addr--4722424c-7012-56b0-84d5-01d076fc547b'}], 'first_name': 'Naive', 'last_name': 'Smith', 'middle_name': 'Weakling', 'prefix': 'Mr', 'social_media_accounts': [{'digital_contact_type': 'work', 'user_account_ref': 'user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b'}], 'team': 'Sales'}}}

laptop_identity-->{'type': 'identity', 'spec_version': '2.1', 'id': 'i

## B Make the Anecdote

In [5]:
# 1. Setup the form paths and the results path
anecdote_form = "SCO/Anecdote/anecdote_on_impact.json"
results_path = "step2/step2"
# 2. Invoke the Make Anecdote block
anecdote = invoke_make_anecdote_block(anecdote_form, results_path, anecdote_reporter=reporter_identity)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
anecdote_results_obj_path = results_base + results_path + "__anecdote.json"
anecdote_results_context_path = results_base + "/step2/anecdote_context.json"
result = invoke_save_incident_context_block(anecdote_results_obj_path, anecdote_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "anecdote",
    "spec_version": "2.1",
    "id": "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f",
    "value": "I clicked on the link, and my laptop screen went weird",
    "report_date": "2024-04-27T06:09:26.291316Z",
    "provided_by_ref": "identity--0ff92ca8-5425-4649-bda9-37c564921944"
}
 result->{'context_result': ' incident context saved -> unattached stix_id -> anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f'}


## C. Collect the Elements of the Observation into an Observed-Data Object

Tbis then represents the potential phishing email as a group of elements

In [6]:
# 1. Setup0 list of obsrvations
observed_ids = []
observed_ids.append(anecdote)
# 2. Setup path to form and results
obs_path ="SDO/Observed_Data/observation-alert.json"
results_path ="step2/observation-anecdote.json"
# 3. Invoke the Make Observed Data Block
obs_2 = invoke_make_observed_data_block(obs_path, results_path, observation=observed_ids)
# 4. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
observation_results_obj_path = results_base + "/step2/observation-anecdote.json"
observation_results_context_path = results_base + "/step2/observation_anecdote_context.json"
result = invoke_save_incident_context_block(observation_results_obj_path, observation_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--b3658639-29e4-45c0-93d3-bfca39180ec9",
    "created": "2024-04-27T06:09:26.319Z",
    "modified": "2024-04-27T06:09:26.319Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f"
    ]
}
 result->{'context_result': ' incident context saved -> unattached stix_id -> observed-data--b3658639-29e4-45c0-93d3-bfca39180ec9'}


## D. Create the Anecdote Sighting

Tbis then represents the Anecdote on Impact from the reporting user

### D.1 Create the Sighting
Connecting the:
- observed-data object
- the reporter identity as the sighted object
- the laptop identity as the location

In [7]:
# 1. Setup path to form and results
sighting_data_path ="SRO/Sighting/sighting_anecdote.json"
results_path ="step2/sighting_anecdote.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing #
observed_list = [obs_2]
sighted = reporter_identity
TR_where_list = [TR_identity]
# 2. Invoke the Make Observed Data Block
sight2 = invoke_sighting_block(sighting_data_path, results_path, observed=observed_list, sighted=sighted, where=TR_where_list)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
sighting_results_obj_path = results_base + "/step2/sighting_anecdote.json"
sighting_results_context_path = results_base + "/step2/sighting_anecdote_context.json"
result = invoke_save_incident_context_block(sighting_results_obj_path, sighting_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sighting",
    "spec_version": "2.1",
    "id": "sighting--fba84581-ff8b-4585-8bd7-44acced80e3c",
    "created": "2024-04-27T06:09:26.353Z",
    "modified": "2024-04-27T06:09:26.353Z",
    "count": 1,
    "sighting_of_ref": "identity--0ff92ca8-5425-4649-bda9-37c564921944",
    "observed_data_refs": [
        "observed-data--b3658639-29e4-45c0-93d3-bfca39180ec9"
    ],
    "where_sighted_refs": [
        "identity--fedbebf0-17fc-4df3-bb5b-6acaedadfa2e"
    ],
    "extensions": {
        "extension-definition--0d76d6d9-16ca-43fd-bd41-4f800ba8fc43": {
            "extension_type": "property-extension"
        },
        "sighting-anecdote": {
            "person_name": "user-report",
            "person_context": "employee",
            "report_submission": "interview"
        }
    }
}
==== sighting--fba84581-ff8b-4585-8bd7-44acced80e3c
 result->{'context_result': ' incident context saved -> unattached stix_id -> sighting--fba84581-ff8b-4585-8bd7-44acced80e3c'}


### D.2 Move Unattached to Other

In [8]:
total_list = observed_ids
total_list.append(obs_2)
total_list.append(TR_identity)
total_list.append(sight2)
total_list.append(reporter_identity)

obs_context_move_path = results_base + "step2/context/context_move.json"
obs_context_move_results = results_base + "step2/context/context_move_results.json"
result = invoke_move_unattached_to_other_block(obs_context_move_path, obs_context_move_results, total_list)

delete element 3, node id anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f
delete element 3, node id observed-data--b3658639-29e4-45c0-93d3-bfca39180ec9
delete element 0, node id identity--fedbebf0-17fc-4df3-bb5b-6acaedadfa2e
==== sighting--fba84581-ff8b-4585-8bd7-44acced80e3c
delete element 0, node id identity--0ff92ca8-5425-4649-bda9-37c564921944


## E. Create the Impact Objects

Tbis Impact represents the effect reported by the user in the Anecdote

In [9]:
# 1. Setup path to form and results
impact_path ="SDO/Impact/anecdote_impact.json"
results_path ="step2/impact_anecdote.json"
# 2. Setup the number of assets impacted
numbers = {"computers-mobile": 1}
impacted = laptop_identity["id"]
# 2. Invoke the Make Observed Data Block
impact_1 = invoke_make_impact_block(impact_path, results_path, impacted_entity_counts=numbers, impacted_refs=impacted, superseded_by_ref=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "impact"
}
impact_results_obj_path = results_base + results_path
impact_results_context_path = results_base + "/step2/impact_anecdote_context.json"
result = invoke_save_incident_context_block(impact_results_obj_path, impact_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "impact",
    "spec_version": "2.1",
    "id": "impact--6d9b3813-98d0-422a-813f-fa7a3bd0c1c9",
    "created": "2024-04-27T06:09:26.556Z",
    "modified": "2024-04-27T06:09:26.556Z",
    "impact_category": "availability",
    "criticality": 99,
    "description": "Laptop is stuffed",
    "end_time": "2024-04-27T06:09:26.556Z",
    "impacted_entity_counts": {
        "computers-mobile": 1
    },
    "impacted_refs": [
        "identity--5cfd3a4c-4995-4b48-a1df-44d2b6ed78cd"
    ],
    "recoverability": "regular",
    "start_time": "2024-04-27T06:09:26.556Z",
    "extensions": {
        "availability": {
            "availability_impact": 99
        },
        "extension-definition--7cc33dd6-f6a1-489b-98ea-522d351d71b9": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> impact stix_id -> impact--6d9b3813-98d0-422a-813f-fa7a3bd0c1c9'}


## F. Create the Next Task Object

Next step is to investigate the Exchange server to see who else got the email

### F.1 Create the Task Object

In [10]:
# 1. Setup path to form and results
task_data_path ="SDO/Task/task_anecdote.json"
results_path ="step2/task_anecdote.json"
# 2. Invoke the Make Observed Data Block
task_2 = invoke_make_task_block(task_data_path, results_path, changed_objects=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "task"
}
task_results_obj_path = results_base + "/step2/task_anecdote.json"
task_results_context_path = results_base + "/step2/task_anecdote_context.json"
result = invoke_save_incident_context_block(task_results_obj_path, task_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "task",
    "spec_version": "2.1",
    "id": "task--d37604fc-e717-4749-8e10-bb7be1078c4e",
    "created": "2024-04-27T06:09:26.616Z",
    "modified": "2024-04-27T06:09:26.616Z",
    "description": "Find out who else got the email",
    "end_time": "2024-04-27T06:09:26.616Z",
    "name": "Query Exchange Server",
    "start_time": "2024-04-27T06:09:26.616Z",
    "extensions": {
        "extension-definition--2074a052-8be4-4932-849e-f5e7798e0030": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> task stix_id -> task--d37604fc-e717-4749-8e10-bb7be1078c4e'}


### F.2 Create the Sequence Object

In [11]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_anecdote.json"
results_path ="step2/sequence_task_anecdote.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_2["id"]
# 2. Invoke the Make Observed Data Block
seq_T_2 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "sequence"
}
sequence_results_obj_path = results_base + "/step2/sequence_task_anecdote.json"
sequence_results_context_path = results_base + "/step2/sequence_anecdote_context.json"
result = invoke_save_incident_context_block(sequence_results_obj_path, sequence_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--e14a5b95-98fe-4050-b18d-c06d22bb7516",
    "created": "2024-04-27T06:09:26.664Z",
    "modified": "2024-04-27T06:09:26.664Z",
    "sequenced_object": "task--d37604fc-e717-4749-8e10-bb7be1078c4e",
    "sequence_type": "task",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> sequence stix_id -> sequence--e14a5b95-98fe-4050-b18d-c06d22bb7516'}


### F.3 Connect the Sequence to the Previous one

In [12]:
# we need to build a block to get the last sequence from memory

## G. Finally, Append all of the objects to the Incident Object

Assign all of the objets to the Incident Object

## H. Write out the Contex Memory for the Incident

Export out the Context Memory for the Incident